In [10]:
import pandas as pd
import numpy as np
from typing import Dict

from PIL import Image
from datasets import load_dataset, Dataset, Features, Value

from sklearn.model_selection import train_test_split

# Data

In [11]:
df2 = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=100, random_state=1)

categories = list(filter(lambda x: x != None, pd.DataFrame([i for i in list(df2['annotations'].apply(lambda x: [i['caption'] for i in x]))]).to_numpy().flatten()))
num_cat = len(categories)
categories = pd.DataFrame(categories)
categories.drop_duplicates(inplace=True)
categories = categories.values.flatten().tolist()
id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

# df2['id'] = list(map(lambda x: {'id':x}, df2.index)) #not correct
df2["image_id"] = df2.index
df2['width'] = df2['image'].apply(lambda x: Image.open('../data/images/' + x).size)
df2['height'] = df2['width'].apply(lambda x: x[1])
df2['width'] = df2['width'].apply(lambda x: x[0])
df2['img'] = df2['image'].apply(lambda x: Image.open('../data/images/' +x))
# df2['height'] = df2['annotations'].apply(lambda x: [i['bbox'][3] for i in x])
df2['area'] = df2['annotations'].apply(lambda x: {"area":[i['bbox'][2] * i['bbox'][3] for i in x]})
df2['bbox'] = df2['annotations'].apply(lambda x: {"bbox":[i['bbox'] for i in x]})
df2['category'] = df2['annotations'].apply(lambda x: {'category':[label2id[i['caption']] for i in x]})
df2 = df2.drop(columns=['annotations'])

counter = 0
lst = []
for i in df2['category']:
	size = len(i['category'])
	each = list(np.arange(counter, counter + size))
	lst.append({'id':each})
	counter += size
df2['id'] = lst

df_vlm = pd.DataFrame({
	"file_name" : df2['image'],
	"image_id" : df2['image_id'],
	"image" : df2['img'],
	"width" : df2['width'],
	"height": df2['height'],
	"objects": df2.apply(lambda x: {'id': x['id']['id'], 'area': x['area']['area'],'bbox': x['bbox']['bbox'], 'category': x['category']['category']}, axis=1)
})
# df_vlm.to_csv('meta_data.csv', index=False)
df_vlm

,file_name,image_id,image,width,height,objects
5074,image_5074.jpg,5074,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [0, 1, 2, 3, 4, 5, 6, 7], 'area': [2640..."
2711,image_2711.jpg,2711,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [8, 9, 10, 11, 12], 'area': [2048, 2496..."
3670,image_3670.jpg,3670,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [13, 14, 15, 16], 'area': [960, 3072, 1..."
2377,image_2377.jpg,2377,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [17, 18, 19, 20, 21, 22, 23], 'area': [..."
4195,image_4195.jpg,4195,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [24, 25, 26, 27, 28, 29, 30], 'area': [..."
...,...,...,...,...,...,...
1771,image_1771.jpg,1771,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [521, 522, 523, 524, 525, 526], 'area':..."
1832,image_1832.jpg,1832,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [527, 528, 529, 530, 531], 'area': [249..."
2289,image_2289.jpg,2289,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [532, 533, 534, 535, 536], 'area': [249..."
2586,image_2586.jpg,2586,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [537, 538, 539, 540, 541, 542, 543, 544..."


In [12]:
len(categories)

123

# Split Dataset

In [13]:
# train, test = train_test_split(df_vlm, test_size=0.2, random_state=1)

# train, val = train_test_split(train, test_size=0.125, random_state=1) # 0.125 x 0.8 = 0.1

df_vlm_csv = df_vlm.drop(columns=['image'])
df_vlm_csv.to_csv('../data/img/metadata.csv', index=False)
pd.DataFrame({
	'name':df_vlm['file_name'],
	'img':df_vlm['image']
}).apply(lambda x: x[1].save('../data/img/' + x[0]), axis=1)

# val_csv = val.drop(columns=['image'])
# val_csv.to_csv('../data/val/metadata.csv', index=False)
# pd.DataFrame({
# 	'name':val['file_name'],
# 	'img':val['image']
# }).apply(lambda x: x[1].save('../data/val/' + x[0]), axis=1)

# test_csv = test.drop(columns=['image'])
# test_csv.to_csv('../data/test/metadata.csv', index=False)
# pd.DataFrame({
# 	'name':test['file_name'],
# 	'img':test['image']
# }).apply(lambda x: x[1].save('../data/test/' + x[0]), axis=1)

# train.shape, val.shape, test.shape

5074    None
2711    None
3670    None
2377    None
4195    None
        ... 
1771    None
1832    None
2289    None
2586    None
4221    None
Length: 100, dtype: object